<a href="https://colab.research.google.com/github/tomonari-masada/course2022-nlp/blob/main/01_bag_of_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **bag-of-wordsモデル**

* この授業全体の参考書
 * 岡﨑、荒瀬、鈴木、鶴岡、宮尾著 『IT Text 自然言語処理の基礎』（オーム社） https://www.ohmsha.co.jp/book/9784274229008/
* 本日の参考書
 * C. D. Manning, P. Raghavan & H. Schütze. Introduction to Information Retrieval. © 2008 Cambridge University Press https://nlp.stanford.edu/IR-book/html/htmledition/irbook.html

* 軽い前置き
 * この授業では、Pythonのコーディングの基礎は習得済みであることを前提します。
 * また、NumPyやscikit-learnの基本的な使い方は習得済みであることを前提します。

## 歴史：bag-of-wordsモデルを取り巻く状況
* bag-of-wordsは、文書をモデル化する方法の、一つ。
* 今は、EDA (exploratory data analysis) 以外ではあまり使われない。
* このbag-of-wordsから、現在のcontextualized word embeddingsに至るまでの流れを把握することが、この授業における学習の目標。
 * LLMs (large language models) は、contextualized word embeddingsを実現する手法の一つ。

### 用語
* 「単語トークン word token」（あるいは単に「トークン token」）
 * 単語の一回一回の出現のこと。
 * このセルで「この」という単語は5回現れている。
 * このことを、このセルでは「この」という単語のトークンが5個ある、などと言い表す。

### 単語のmultisetとしての文書
* **bag-of-wordsモデル**とは、文書をベクトルとしてモデル化する手法のひとつ。
 * 他にも文書をベクトル化する手法はある。
* bag-of-wordsモデルにおいては、文書における単語トークンの**出現順序が無視される**。
* つまり、文書を、バッグに入ったアイテムの集まりのようにモデリングする（下図参照）。
 * 言い換えれば、文書を単語の**multiset**として扱うのがbag-of-wordsモデルである。

* 参考資料
 * https://github.com/aws-samples/aws-machine-learning-university-accelerated-nlp/blob/master/notebooks/MLA-NLP-Lecture1-BOW.ipynb



![bag-of-words.png](https://raw.githubusercontent.com/tomonari-masada/course2022-nlp/main/bag-of-words.png)

* 図は下記のWebページより。
 * https://dudeperf3ct.github.io/lstm/gru/nlp/2019/01/28/Force-of-LSTM-and-GRU/

### 単語のベクトル表現の隆盛
* 最近では、言語データをモデル化するとき、単語（あるいはsubword）のベクトル表現を用いる。
* このベクトル表現は、埋め込みembeddingと呼ばれる。
* word embeddingやsubword embeddingを使うのが、今は主流。
 * document embeddingも、word embeddingをもとにして構成する。

### 今となってはobsoleteなbag-of-wordsモデル
* 論文では今でも、baselineとして、TF-IDFやBM25など、bag-of-wordsモデルが引き合いに出されることはある。
 * 新しい手法を考え出しても、bag-of-wordsに勝てなければ意味がない、といった使い方。
* 参考 
 * https://twitter.com/moguranosenshi/status/1306406087445196800
 * https://twitter.com/sho_yokoi/status/1553044631864360960
 * https://twitter.com/odashi_t/status/1552951268842545154
* そのため、授業の最初に、bag-of-wordsモデルについて簡単に説明しておく。
 * BM25 https://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html

### 自然言語処理の歴史
* 興味がある方は、スタンフォード大の自然言語処理の授業が、この10年間でいかに大きく内容を変えているか、調べてみましょう。

 * http://web.stanford.edu/class/cs224n/index.html の"Previous offerings"

## binary vector
* 最も単純には、文書は、各単語が出現するかしないかの、1/0の2値ベクトルでモデル化できる。

### 用語
* 「語彙 vocabulary」
 * あるデータセットに出現する単語の集合のこと。

### scikit-learnのCountVectorizer
* 各documentは、半角スペースでつながれた単語の列として準備しておく。
* binary=Trueとすると、0/1の2値ベクトルが得られる。
* インスタンスを作り、fit_transformする、という使い方は、scikit-learnにおけるデータの前処理のときと同様。

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

* 文書の集合＝コーパスを用意する。

In [ ]:
corpus = ["This document is the first document.",
          "This document is the second document.",
          "And this is the third one.",
          "Where is the fourth one?"]

* CountVectorizerをbinary=Trueで使う

In [ ]:
binary_vectorizer = CountVectorizer(binary=True) # 2値ベクトルとして表現
X = binary_vectorizer.fit_transform(corpus)

* 文書の2値ベクトル表現の確認
 * 疎なベクトルとして得られることに注意。

In [ ]:
print(X)

In [ ]:
type(X)

* 疎な表現を通常のndarrayに戻すには・・・

In [ ]:
X.toarray()

### 語彙を確認
* 先頭の大文字は自動的に小文字に変換されていることが分かる。
* ピリオドや疑問符は削除されている。

In [ ]:
binary_vectorizer.vocabulary_

In [ ]:
type(binary_vectorizer.vocabulary_)

In [ ]:
print(binary_vectorizer.get_feature_names_out())

### 新しい文書をベクトルに変換
* sklearnでよくやるように、transformメソッドを使う。

In [ ]:
new_doc = ["This is the new document."]

new_vectors = binary_vectorizer.transform(new_doc)

* 新出の単語は無視される点に注意
 * OoV (out-of-vocabulary) wordsの問題
 * この問題は、NLPの世界では、超重要な問題。
 * 今は、subwordの利用により、OoV問題を回避する。

* newという単語は、無視されている。

In [ ]:
new_vectors.toarray()

## word count vector
* 文書における各単語の出現回数を使って、文書のベクトル表現を得ることもできる。

### scikit-learnのCountVectorizer
* CountVectorizerをデフォルト設定で（binary=Trueとせずに）使う
* すると、単語の出現回数による文書のベクトル表現が得られる

In [ ]:
count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform(corpus)

In [ ]:
X.toarray()

In [ ]:
new_vectors = count_vectorizer.transform(new_doc)
print(new_vectors.toarray())

## TF-IDF
* 文書をベクトル化する古典的な手法。
* TF-IDFは、TFとIDFの積である。

### TF (term frequency)
* 文書に含まれる単語トークンの数（つまり、単語の出現回数の総和）を、その文書の長さと呼ぶ。
* TFとは、各々の単語が文書のなかで出現する回数を、その文書の長さで割ったもの。
 * 文書のなかで頻出する単語ほどTFは大きくなる。

### IDF (inverse document frequency)
* IDFとは、DFの逆数。
* DFとは、ある単語が含まれる文書の数を、総文書数で割ったものである。
 * 文書集合のなかで稀少な単語ほどIDFは大きくなる。

### TF-IDF (term frequency–inverse document frequency)
* TF-IDFは、TFとIDFの積。
* 積を求める前に、TFのルートもしくは対数をとったり、IDFのルートもしくは対数をとったりする。
 * 大きめの値が、効きすぎないようにする。
 * 対数をとるときは、ゼロの対数をとることにならないような工夫をする。

### TF-IDFの式の例

\begin{align}
x_{d,w} = \frac{n_{d,w}}{n_d} \cdot ( 1 + \ln\frac{m}{m_w}) \tag{1}
\end{align}

where 

 * $n_{d,w}$ is the frequency of the word $w$ in the document $d$, 
 * $n_d$ is defined as $n_d \equiv \sum_w n_{d,w}$,
 * $m_w$ is the number of documents containing the word $w$, and
 * $m$ is the total number of documents.

### TF-IDFの式のバリエーション

![img462.png](https://raw.githubusercontent.com/tomonari-masada/course2022-nlp/main/img462.png)

https://nlp.stanford.edu/IR-book/html/htmledition/document-and-query-weighting-schemes-1.html

* 式の選び方
  * どの式の形がいいかは、downstream taskの性能をcross validationで評価して選ぶ。
  * どんな場合でもこれが一番、という式は、ない。

### scikit-learnのTfidfVectorizer
* scikit-learnでのTF-IDFの計算式がどうなっているかは、下記ページを参照。
 * https://scikit-learn.org/stable/modules/feature_extraction.html#tfidf-term-weighting

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()

* デフォルトの設定を確認してみる。
 * 次のパラメータは変更していいかもしれない。
 * max_df, min_df, ngram_range, norm, smooth_idf, stop_words, sublinear_tf

In [ ]:
tfidf_vectorizer

In [ ]:
X = tfidf_vectorizer.fit_transform(corpus).toarray()
print(X)

* 文書ベクトルはL2ノルムが1となるように長さを変更されている。
 * TfidfVectorizer()のnormパラメータで変更可能。


In [ ]:
import numpy as np
np.linalg.norm(X, axis=1)

In [ ]:
tfidf_vectorizer.get_feature_names_out()

In [ ]:
new_vectors = tfidf_vectorizer.transform(new_doc).toarray()
print(new_vectors)

* 各単語のIDF
 * IDFはそれぞれの単語について一意に決まる値。
 * 文書ごとに求まる値ではない。
 * コーパスが変わると、IDFも変わる。

In [ ]:
tfidf_vectorizer.idf_

## bag-of-wordsベクトルの応用
* 文書間の類似度の計算に使える

* sklearnでは、ベクトルが長さ1にnormalizeされている。
* そのため、内積がコサイン類似度に一致する。

In [ ]:
for i in range(4):
  print(np.dot(X[i], new_vectors[0]))

# 課題1
* 文書をベクトルとして表現する方法が分かった。
* これを使うと、何ができるか？

## 20 newsgroups データセット
* 文書分類手法の評価に使う、古典的なデータセット。

In [ ]:
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups()
y_true = newsgroups.target

* 下記コードを参考にして、数値を全て「#NUMBER」という特殊な単語へ変換する。
 * https://scikit-learn.org/stable/auto_examples/bicluster/plot_bicluster_newsgroups.html#sphx-glr-auto-examples-bicluster-plot-bicluster-newsgroups-py

In [ ]:
def number_normalizer(tokens):
    """ Map all numeric tokens to a placeholder.

    For many applications, tokens that begin with a number are not directly
    useful, but the fact that such a token exists can be relevant.  By applying
    this form of dimensionality reduction, some methods may perform better.
    """
    return ("#NUMBER" if token[0].isdigit() else token for token in tokens)


class NumberNormalizingVectorizer(TfidfVectorizer):
    def build_tokenizer(self):
        tokenize = super().build_tokenizer()
        return lambda doc: list(number_normalizer(tokenize(doc)))

In [ ]:
vectorizer = NumberNormalizingVectorizer(stop_words='english', min_df=5)

In [ ]:
X = vectorizer.fit_transform(newsgroups.data)

In [ ]:
print(vectorizer.get_feature_names_out()[:20])

In [ ]:
len(vectorizer.get_feature_names_out())

In [ ]:
X = X.toarray()

In [ ]:
np.dot(X[0], X[1])

In [ ]:
y_true

In [ ]:
newsgroups.target_names